In [ ]:
import pandas as pd
import pandas as pd
import nltk
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
nltk.download('wordnet')


# make sure you imported the kaggle dataset and have it on the same directory as this.
file = "wiki_movie_plots_deduped.csv"

df = pd.read_csv(file)
df

[nltk_data] Downloading package wordnet to /root/nltk_data...


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
...,...,...,...,...,...,...,...,...
34881,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ..."
34882,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the..."
34883,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i..."
34884,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
condition = df['Genre'] == "romantic comedy"
romantic_df = df[condition]
romantic_df



In [ ]:
# This cell takes awhile to run

selected_col = df[["Genre", "Plot"]] # columns from the csv that we want to focus on

selected_genres = ["comedy", "drama", "adventure", "fantasy",
                  "horror", "mystery", "romance",
                 "action", "thriller", "western"] # select 10 genres

# Initialize an empty dictionary
movie = {}

# Iterate through the DataFrame rows
for index, row in selected_col.iterrows():
    genre = row['Genre']
    plot = row['Plot']

    # Check if genre exists in selected_genres
    if genre in selected_genres:
        if genre in movie:
            # Append the movie plot to the existing value
            # I did < 250 to ensure that every genre has less than 250 movies. This is because we had too much
            # text so I had to shorten it
            if len(movie[genre]) < 250:
                movie[genre].append(plot)
        else:
            # Create a new key-value pair
            movie[genre] = [plot]



# Right now, the result of movie looks like this:
# movie = {comedy: [movie1, movie2, movie3,...,movie250], action: [movie1, movie2, movie3,...,movie250]} and so on

# So the value of the key-value pairs is just a list of the movies

selected_col

,Genre,Plot
0,unknown,"A bartender is working at a saloon, serving dr..."
1,unknown,"The moon, painted with a smiling face hangs ov..."
2,unknown,"The film, just over a minute long, is composed..."
3,unknown,Lasting just 61 seconds and consisting of two ...
4,unknown,The earliest known adaptation of the classic f...
...,...,...
34881,unknown,"The film begins in 1919, just after World War ..."
34882,comedy,"Two musicians, Salih and Gürkan, described the..."
34883,comedy,"Zafer, a sailor living with his mother Döndü i..."
34884,romantic comedy,The film centres around a young woman named Am...


In [ ]:
# print total amount of words in entire text corpus that we are analyzing
total_words = 0
for key in movie:
    for mov in movie[key]:
        total_words += len(mov.split(' '))
print(total_words)

849167


In [ ]:
# prints out the total amount of movies we are analyzing

total_movies = 0
for key in movie:
    total_movies += len(movie[key])
print(total_movies)

2454


In [ ]:
# So to get the text corpus for a particular genre, do something like:

# text = movie["comedy"]
# because this 'text' variable is a list (because the value from the key-value pair is a list)
# we want to make it a string
# so do something like new_text = "".join(text)
# get rid of stop words, normalize, lemmatize, etc, and make visualization from there

text = movie["comedy"]
#print(type(text))
new_text = "".join(text)
#print(type(new_text))

# get stop words
# tokenize/lemmatize depending on what your doing

In [ ]:
def uniBigrams(text):
  stoplist = stopwords.words('english')
  stoplist.extend([".", ",", "?", "could", "would", "“", "”", "’", ";", "!", "(", ")", "'s", "[", "]", '``', "''"])

  alltokens = nltk.word_tokenize(text)
  allcontenttokens = [w for w in alltokens if w.lower() not in stoplist]

  from nltk.stem import WordNetLemmatizer
  lemmatizer = WordNetLemmatizer()

  # create a new list of tokens, alllemmas by lemmatizing allcontenttokens
  all_lemmas = [lemmatizer.lemmatize(w) for w in allcontenttokens]

  unigrams = nltk.ngrams(all_lemmas, 1)
  unigramlist = list(unigrams)

  bigrams = nltk.ngrams(all_lemmas, 2)
  bigramlist = list(bigrams)

  # print out most frequent bigrams and unigrams
  bigramfreq = nltk.FreqDist(bigramlist)
  unigramfreq = nltk.FreqDist(unigramlist)
  maxBigrams = bigramfreq.most_common(5)
  maxUnigrams = unigramfreq.most_common(20)
  return [maxBigrams, maxUnigrams]



In [ ]:
#Bigram finder
for genre in selected_genres:
  text = movie[genre]
  new_text = "".join(text)
  unibigram = uniBigrams(new_text)
  actualBigram = unibigram[0]
  print(genre, actualBigram)



comedy [(('young', 'man'), 40), (('young', 'woman'), 25), (('fall', 'love'), 25), (('New', 'York'), 18), (('Mrs.', 'Erlynne'), 17)]
drama [(('described', 'film'), 60), (('film', 'magazine'), 57), (('fall', 'love'), 41), (('New', 'York'), 34), (('magazine', '3'), 21)]
adventure [(('fall', 'love'), 31), (('Johnny', 'Pacific'), 23), (('United', 'States'), 19), (('next', 'day'), 17), (('de', 'Maynes'), 17)]
fantasy [(('Little', 'Bear'), 37), (('next', 'day'), 33), (('fall', 'love'), 30), (('White', 'Snake'), 28), (('Green', 'Snake'), 24)]
horror [(('Lady', 'Jane'), 62), (('next', 'day'), 49), (('next', 'morning'), 46), (('Sir', 'Frederick'), 40), (('Van', 'Helsing'), 28)]
mystery [(('Lone', 'Wolf'), 40), (('New', 'York'), 32), (('Holmes', 'Watson'), 23), (('found', 'dead'), 22), (('Los', 'Angeles'), 21)]
romance [(('New', 'York'), 76), (('fall', 'love'), 62), (('next', 'day'), 46), (('York', 'City'), 33), (('next', 'morning'), 25)]
action [(('United', 'States'), 39), (('New', 'York'), 36),

In [ ]:
def getText(genre):
  text = movie[genre]
  new_text = "".join(text)
  return new_text

In [ ]:
def tokens(text):
  stoplist = stopwords.words('english')
  stoplist.extend([".", ",", "?", "could", "would", "“", "”", "’", ";", "!", "(", ")", "'s", "[", "]", '``', "''"])

  alltokens = nltk.word_tokenize(text)
  allcontenttokens = [w for w in alltokens if w.lower() not in stoplist]

  from nltk.stem import WordNetLemmatizer
  lemmatizer = WordNetLemmatizer()

  # create a new list of tokens, alllemmas by lemmatizing allcontenttokens
  all_lemmas = [lemmatizer.lemmatize(w) for w in allcontenttokens]

  return all_lemmas

In [ ]:
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

def collocations(genre):
  text = getText(genre)
  filtered_text = tokens(text)

  bigram_finder = BigramCollocationFinder.from_words(filtered_text)
  bigram_finder.apply_freq_filter(3)  # Adjust the frequency filter as needed

  collocations = bigram_finder.nbest(BigramAssocMeasures.likelihood_ratio, 10)

  # Print Collocations
  print("Genre:" + genre)
  print("Collocations:", collocations[0:3])

In [ ]:
for genre in selected_genres:
  collocations(genre)

Genre:comedy
Collocations: [('young', 'man'), ('New', 'York'), ('Mrs.', 'Erlynne')]
Genre:drama
Collocations: [('described', 'film'), ('film', 'magazine'), ('New', 'York')]
Genre:adventure
Collocations: [('United', 'States'), ('Johnny', 'Pacific'), ('fall', 'love')]
Genre:fantasy
Collocations: [('Little', 'Bear'), ('White', 'Snake'), ('Green', 'Snake')]
Genre:horror
Collocations: [('Lady', 'Jane'), ('Sir', 'Frederick'), ('next', 'morning')]
Genre:mystery
Collocations: [('Lone', 'Wolf'), ('New', 'York'), ('Los', 'Angeles')]
Genre:romance
Collocations: [('New', 'York'), ('fall', 'love'), ('York', 'City')]
Genre:action
Collocations: [('United', 'States'), ('Los', 'Angeles'), ('New', 'York')]
Genre:thriller
Collocations: [('New', 'York'), ('Los', 'Angeles'), ('next', 'day')]
Genre:western
Collocations: [('Gene', 'Autry'), ('Smiley', 'Burnette'), ('Millhouse', 'Smiley')]
